# Ensemble

### Ensemble Learning
- 여러 개의 기본 모델을 활용하여 하나의 새로운 모델을 만들어내는 개념
- 이 때 기본모델을 weak learner, classifier, base learner, singler learner이라고 부름.

- 이진 분류 문제에서 각각의 base learner들을 0.5 이상의 정확도를 가져야함.
- 각각의 분류기는 독립이어야함.
- 총 분류기의 수는 이론적으로 무한대이어야함.

- Test 데이터에 대해 다양한 의견(예측값)을 수렴하기 위해 overfitting이 잘 되는 모델 사용
- Ensemble 개념 자체는 여러 모델의 조합을 뜻하기에 Tree가 아닌 다른 모델 사용 가능
- 가장 많이 쓰이는 RandomForest, Boosting은 Tree기반 모델.

### 대수의 법칙
- 큰 모집단에서 무작위로 뽑은 표본의 평균이 전체 모집단의 평균과 가까울 가능성이 높다
- 많은 시행(다수의 사람)의 결과가 수학적(이성적)으로 합리적인 결과를 가져온다.

# Ensemble의 종류

# --------- Tree 기반의 단일 모델 ------------

### Bagging
- 모델을 다양하게 만들기 위해 데이터를 재구성
- 데이터의 행을 조정

### RandomForest 
- 모델을 다양하게 만들기 위해 데이터 뿐만 아니라 변수도 재구성
- 데이터의 행과 열을 조정

### Boosting
- 맞추기 어려운 데이터에 대해 좀 더 가중치를 두어 학습하는 개념

# --------- Ensemble 한 개념 ------------

### Stacking
- 모델의 output값을 새로운 독립변수로 사용

<br>
<br>

# Bagging(Bootstrap aggregating)
### Bootstrapping

- 복원추출의 의미. 10개의 공 중 1개 씩 5번 넣고 빼고 할 때 나오는 샘플.

### Tree vs Bagging
- 깊이 성장한 트리 : 분산 증가, 편향 감소
- Bagging : 트리의 편향 유지, 분산 감소 / 학습데이터의 노이즈에 강건해짐 / 모형해석의 어려움

In [4]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [16]:
data = pd.read_csv('./data/kc_house_data.csv')
data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180.0,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170.0,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770.0,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050.0,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680.0,0,1987,0,98074,47.6168,-122.045,1800,7503


In [17]:
ncar = data.shape[0]
nvar = data.shape[1]
print(ncar, nvar)

21613 21


### 의미 없는 변수 제거

In [19]:
data = data.drop(['id', 'view', 'date', 'zipcode', 'lat', 'long', 'sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement', 'sqft_living15', 'sqft_lot15'],axis=1)
data.head()

,price,bedrooms,bathrooms,floors,waterfront,condition,grade,yr_built,yr_renovated
0,221900.0,3,1.00,1.0,0,3,7,1955,0
1,538000.0,3,2.25,2.0,0,3,7,1951,1991
2,180000.0,2,1.00,1.0,0,3,6,1933,0
3,604000.0,4,3.00,1.0,0,5,7,1965,0
4,510000.0,3,2.00,1.0,0,3,8,1987,0


### 범주형 변수를 이진형 변수로 변환
- 범주형 변수는 waterfront 뿐이며, 이진 분류이기 때문에 0, 1로 표현.
- 데이터에서 0, 1로 표현되어 있으므로 생략

### 설명변수와 타겟변수를 분리, 학습데이터와 평가데이터 분리

In [20]:
feature_columns = list(data.columns.difference(['price']))
X = data[feature_columns]
y = data['price']
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.3, random_state=42)
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)

(15129, 8) (6484, 8) (15129,) (6484,)


### 학습 데이터를 선형 회귀 모형에 적합 후 평가 데이터로 검증(Stats_Models)

In [23]:
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

In [24]:
sm_train_x = sm.add_constant(train_x, has_constant='add')
sm_model = sm.OLS(train_y, sm_train_x)
fitted_sm_model = sm_model.fit()
fitted_sm_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.595
Model:                            OLS   Adj. R-squared:                  0.595
Method:                 Least Squares   F-statistic:                     2775.
Date:                Thu, 18 Mar 2021   Prob (F-statistic):               0.00
Time:                        11:10:48   Log-Likelihood:            -2.0825e+05
No. Observations:               15129   AIC:                         4.165e+05
Df Residuals:                   15120   BIC:                         4.166e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const         7.181e+06   1.73e+05     41.543      0.000    6.84e+06    7.52e+06
bathrooms     1.302e+05   3958.626     32.885      0.000    1.22e+05    1.38e+05
bedrooms     -2208.0602   2381.029     -0.927      0.354   -6875.165    2459.045
condition     1.639e+04   3167.247      5.174      0.000    1.02e+04    2.26e+04
floors        1961.5736   4334.423      0.453      0.651   -6534.419    1.05e+04
grade         1.954e+05   2198.315     88.907      0.000    1.91e+05       2e+05
waterfront    7.551e+05   2.26e+04     33.476      0.000    7.11e+05    7.99e+05
yr_built     -4297.7410     88.024    -48.825      0.000   -4470.278   -4125.204
yr_renovated    12.7222      5.040      2.524      0.012       2.842      22.602
==============================================================================
Omnibus:                    13460.787   Durbin-Watson:                   1.994
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1692932.643
Skew:                           3.768   Prob(JB):                         0.00
Kurtosis:                      54.272   Cond. No.                     1.82e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.82e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

- R-squared - y의 총 변동성 중 x가 설명하는 변동의 비율 : 0.595
    > x 들을 가지고 집값을 설명하는 변동의 비율이 약 59.5%다.

- 집 안에 화장실 개수가 많을 수록 비싸다는 것을 의미.
- 반면에, 침실이 많을 수록 집값이 떨어지는 결과가 나오는데 p-value가 높게 나옴
    > 귀무가설 채택.
    
- bathroom과 bedroom 사이에 다중공선성이 존재하여 그런 것이 아닐까라는 생각.

In [26]:
sm_test_x = sm.add_constant(test_x, has_constant='add')
sm_model_predict = fitted_sm_model.predict(sm_test_x)


In [29]:
sqrt(mean_squared_error(sm_model_predict, test_y)) # RMSE

239648.51635185885

### Bagging 한 결과가 일반적인 결과보다 좋은지 확인

- for 문을 사용하여 구하기

In [38]:
# 복원추출
bagging_predict_result = []
for _ in range(10):
    data_index = [data_index for data_index in range(train_x.shape[0])]
    random_data_index = np.random.choice(data_index, train_x.shape[0])
    print(len(set(random_data_index)))
    sm_train_x = train_x.iloc[random_data_index,]
    sm_train_y = train_y.iloc[random_data_index,]
    sm_train_x = sm.add_constant(sm_train_x, has_constant='add')
    sm_model = sm.OLS(sm_train_y, sm_train_x)
    fitted_sm_model = sm_model.fit()
    pred = fitted_sm_model.predict(sm_test_x)
    bagging_predict_result.append(pred)
    print(sqrt(mean_squared_error(pred, test_y)))

9587
239497.5465684638
9473
240009.37312881128
9576
240894.18607079834
9665
239643.74752200628
9538
240022.83156022575
9452
240235.77498514665
9539
239789.58251235983
9590
239734.69109633085
9583
239728.37414794788
9562
239531.45388108675


In [46]:
print(bagging_predict_result[1][0],
bagging_predict_result[2][0], bagging_predict_result[3][0])
# 이 3가지 예측값을 더해서 평균을 내야 최종적인 앙상블 예측값이 나옴. 

327109.1472448595 320535.4505101871 321729.6916269716


In [48]:
bagging_predict = []
for lst2_index in range(test_x.shape[0]):
    temp_predict = []
    for lst_index in range(len(bagging_predict_result)):
        temp_predict.append(bagging_predict_result[lst_index].values[lst2_index])
    bagging_predict.append(np.mean(temp_predict))

In [52]:
sqrt(mean_squared_error(bagging_predict, test_y)) 
# Bagging을 했을 때 앙상블의 RMSE

239715.8303351919

### 학습 데이터를 선형회귀 모형에 적합 후 평가 데이터로 검증(Scikit-Learn)

In [61]:
from sklearn.linear_model import LinearRegression
regr_model = LinearRegression()
linear_model1 = regr_model.fit(train_x, train_y)

### Bagging을 이용하여 선형회귀모형에 적합 후 평가(Sampling 10번)

In [62]:
from sklearn.ensemble import BaggingRegressor
bagging_model = BaggingRegressor(base_estimator = regr_model, n_estimators=10)
linear_model2 = bagging_model.fit(train_x, train_y)
predict2 = linear_model2.predict(test_x)

print(sqrt(mean_squared_error(predict2, test_y)))

239889.28228744792


- for 문을 돌렸을 때와 비슷한 결과를 보임.

### Sampling 많이 해보기

In [63]:
from sklearn.ensemble import BaggingRegressor
bagging_model = BaggingRegressor(base_estimator = regr_model, n_estimators=100)
linear_model2 = bagging_model.fit(train_x, train_y)
predict2 = linear_model2.predict(test_x)

print(sqrt(mean_squared_error(predict2, test_y)))

239655.3568765973


- 결과가 별반 다르지 않다..

### 학습 데이터를 의사결정나무 모형에 적합 후 평가 데이터로 검증

In [64]:
from sklearn.tree import DecisionTreeRegressor
decision_tree_model = DecisionTreeRegressor()
tree_model = decision_tree_model.fit(train_x, train_y)
predict_tree = tree_model.predict(test_x)
print(sqrt(mean_squared_error(predict_tree, test_y)))

297873.5783368525


- 좋지 않은 결과가 나와 for 문을 이용하여 다시 해보기

In [78]:
# 복원추출
bagging_predict_result = []
for _ in range(30):
    data_index = [data_index for data_index in range(train_x.shape[0])]
    random_data_index = np.random.choice(data_index, train_x.shape[0])
    print(len(set(random_data_index)))
    sm_train_x = train_x.iloc[random_data_index,]
    sm_train_y = train_y.iloc[random_data_index,]
    decision_tree_model = DecisionTreeRegressor()
    tree_model = decision_tree_model.fit(sm_train_x, sm_train_y)
    predict_tree = tree_model.predict(test_x)
    bagging_predict_result.append(predict_tree)
    print(sqrt(mean_squared_error(predict_tree, test_y)))

9474
286451.1727451921
9530
278344.20219988714
9584
280053.8448330828
9628
296362.4066518797
9527
283531.4478777692
9568
280811.06315772195
9602
286899.92621499096
9537
288078.87401098764
9483
276459.8060361904
9623
305237.8166666953
9619
296941.6291716926
9555
286665.60718538606
9557
315440.18365626066
9509
298346.51129853283
9586
287671.284780392
9581
286643.48376749305
9508
297562.755589105
9611
302743.6797854958
9558
291672.2255730762
9579
282329.0835207778
9586
282792.874379605
9561
346117.31253860286
9507
303733.2967609429
9538
301273.3343774617
9533
296541.5399833843
9573
302250.28049837955
9628
301635.817761722
9551
280983.57717414
9518
315710.78061820456
9582
269061.0522408295


In [79]:
bagging_predict = []
for lst2_index in range(test_x.shape[0]):
    temp_predict = []
    for lst_index in range(len(bagging_predict_result)):
        temp_predict.append(bagging_predict_result[lst_index][lst2_index])
    bagging_predict.append(np.mean(temp_predict))

In [80]:
sqrt(mean_squared_error(bagging_predict, test_y))

233743.41043299157

- 이전보다 좋은 모습을 보임.

### Bagging을 이용하여 의사결정나무 모형에 적합 후 평가 (Sampling=10)

In [74]:
bagging_model = BaggingRegressor(base_estimator = decision_tree_model, n_estimators=10)
linear_model2 = bagging_model.fit(train_x, train_y)
predict2 = linear_model2.predict(test_x)

print(sqrt(mean_squared_error(predict2, test_y)))

238138.66497488949


- Sampling = 30

In [84]:
bagging_model = BaggingRegressor(base_estimator = decision_tree_model, n_estimators=30)
linear_model2 = bagging_model.fit(train_x, train_y)
predict2 = linear_model2.predict(test_x)

print(sqrt(mean_squared_error(predict2, test_y)))

232007.31697609698


- 성능이 어느정도 올라가면 더 좋아지지는 않는다.

# Random Forest

- 다양한 모델.
- 모델의 분산을 줄여 일반적으로 Bagging보다 성능이 좋음.
- 데이터 뿐만 아니라, 변수도 랜덤하게 뽑아서 다양한 모델을 만들자!
    > base learner 간의 공분산을 줄이자.
    
    
- 뽑을 변수는 사용자가 지정해줘야하는 hyper parameter.
    > 일반적으로는 sqrt(p) : 루트 p개 사용.

In [86]:
data = pd.read_csv('./data/otto_train.csv')
data.head()

,id,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,...,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93,target
0,1,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,Class_1
1,2,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,Class_1
2,3,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,Class_1
3,4,1,0,0,1,6,1,5,0,0,...,0,1,2,0,0,0,0,0,0,Class_1
4,5,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,Class_1


In [87]:
'''
id : 고유 아이디
feat_1 ~ feat_93 : 설명 변수
target : 타겟변수(1~9)
'''

'\nid : 고유 아이디\nfeat_1 ~ feat_93 : 설명 변수\ntarget : 타겟변수(1~9)\n'

In [88]:
nCar = data.shape[0] # 데이터의 갯수
nVar = data.shape[1] # 변수의 갯수
print(nCar, nVar)

61878 95


### 의미없는 변수 제거

In [89]:
data = data.drop(['id'], axis=1)
data.head()

,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,feat_10,...,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93,target
0,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,Class_1
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Class_1
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Class_1
3,1,0,0,1,6,1,5,0,0,1,...,0,1,2,0,0,0,0,0,0,Class_1
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,Class_1


### 타겟 변수의 문자열을 숫자로 변환

In [95]:
# 타겟변수의 형태를 변환
mapping_dict = {'Class_1' : 1,
                'Class_2' : 2,
                'Class_3' : 3,
                'Class_4' : 4,
                'Class_5' : 5,
                'Class_6' : 6,
                'Class_7' : 7,
                'Class_8' : 8,
                'Class_9' : 9,}
after_mapping_target = data['target'].apply(lambda x : mapping_dict[x])
after_mapping_target[0:4]

0    1
1    1
2    1
3    1
Name: target, dtype: int64

### 설명변수와 타겟변수를 분리, 학습데이터와 평가데이터 분리

In [96]:
feature_columns = list(data.columns.difference(['target']))
X = data[feature_columns]
y = after_mapping_target
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.2, random_state = 42)
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)

(49502, 93) (12376, 93) (49502,) (12376,)


### 학습 데이터를 랜덤포레스트 모형에 적합 후 평가 데이터로 검증

In [97]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [98]:
clf = RandomForestClassifier(n_estimators=20, max_depth=5, random_state=0)
clf.fit(X, y)

RandomForestClassifier(max_depth=5, n_estimators=20, random_state=0)

In [99]:
predict1 = clf.predict(test_x)
print(accuracy_score(test_y, predict1))

0.5968002585649644


### 트리를 많이 만든다면 어떨까?

In [101]:
clf2 = RandomForestClassifier(n_estimators=289, max_depth=5, random_state=0)
clf2.fit(X, y)
predict2 = clf2.predict(test_x)
print(accuracy_score(test_y, predict2))

0.6120717517776342


### 트리의 깊이를 늘리면 어떨까?

In [102]:
clf3 = RandomForestClassifier(n_estimators=20, max_depth=25, random_state=0)
clf3.fit(X, y)
predict3 = clf3.predict(test_x)
print(accuracy_score(test_y, predict3))

0.9278442146089205


In [103]:
clf4 = RandomForestClassifier(n_estimators=200, max_depth=25, random_state=0)
clf4.fit(X, y)
predict4 = clf4.predict(test_x)
print(accuracy_score(test_y, predict4))

0.9330963154492566


- estimator를 늘렸을 때 보다 깊이를 늘리는 것이 정확도가 더 높다!

### 트리의 깊이를 최대로 늘려보기

In [104]:
clf3 = RandomForestClassifier(n_estimators=20, max_depth=99, random_state=0)
clf3.fit(X, y)
predict3 = clf3.predict(test_x)
print(accuracy_score(test_y, predict3))

0.9985455720749838


In [116]:
clf3 = RandomForestClassifier(n_estimators=20, max_depth=66, random_state=0)
clf3.fit(X, y)
predict3 = clf3.predict(test_x)
print(accuracy_score(test_y, predict3))

0.9991111829347123


- 깊이를 66으로 했을 때 정확도가 비약적으로 상승했다!

### 트리의 깊이도 늘리고, 갯수도 늘리기!

In [118]:
clf3 = RandomForestClassifier(n_estimators=100, max_depth=66, random_state=0)
clf3.fit(X, y)
predict3 = clf3.predict(test_x)
print(accuracy_score(test_y, predict3))

1.0


- 1의 정확도를 보임.

# Boosting

- 오분류된 데이터에 초점을 맞추어 더 많은 가중치를 주는 방식
- 초기에는 전 데이터가 동일한 가중치를 갖지만, 각 round가 종료된 후 가중치와 중요도를 계산
- 복원 추출 시에 가중치 분포를 고려
- 오분류된 데이터가 가중치를 더 얻게 됨에 따라 다음 round에서 더 많이 고려됨.

### 1) Gradient Boosting의 종류
- Boosting 기법들 간 차이는 오분류된 데이터를 다음 Round에서 어떻게 반영할 것인가의 차이
- AdaBoost는 오분류된 데이터들에 더 큰 가중치를 주어 다음 Round 샘플링에 반영

    > 즉, 잔차를 예측해서 Boosting을 시킨다.
    
    > 잔차를 계속해서 예측하면 에러를 없앨 수 있다.

### 1-1) XGBoost
- XGBoost는 Gradient Boosting 개념을 의사결정나무에 도입한 알고리즘.
- 데이터 별 오류를 다음 Round 학습에 반영시킨다는 측면에서 기존과 큰 차이는 없음.
- 다만, 학습을 위한 목적식에 Regularization term이 추가되어 과적합을 방지해줌.
- Regularization term을 통해 XGBoost는 복잡한 모델에 패널티를 부여.

- 회귀분석의 Loss Function : SSE를 최소화시키는 것
    > 회귀계수를 축소시키는 항(term) : Regularization term

### 1-2) LightGBM
- xgboost와 다르게 leaf-wise loss 사용(loss를 더 줄일 수 있음)
- xgboost 대비 2배 이상 빠른 속도, GPU 지원
- Overfitting에 민감하여, 대량의 학습데이터 필요

### 1-3) Catboost - categorical features
- 잔차 추정의 분산을 최소로 하면서 bias를 피하는 boosting 기법
- 관측치를 포함한 채로 boosting하지 말고, 관측치를 뺀 채로 학습하여 그 관측치에 대한 unbiased residual을 구하고 학습하자는 아이디어
- Categorical features가 많은 경우 잘 맞는다고 알려져있음.
- Categorical feature를 one-hot encoding 방식이 아니라, 수치형으로 변환하는 방법 제안.

# Ada_Boost 사용해보기

In [121]:
data = pd.read_csv('./data/otto_train.csv')
data.head()

,id,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,...,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93,target
0,1,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,Class_1
1,2,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,Class_1
2,3,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,Class_1
3,4,1,0,0,1,6,1,5,0,0,...,0,1,2,0,0,0,0,0,0,Class_1
4,5,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,Class_1


In [122]:
data = data.drop(['id'], axis=1)
data.head()

,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,feat_10,...,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93,target
0,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,Class_1
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Class_1
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Class_1
3,1,0,0,1,6,1,5,0,0,1,...,0,1,2,0,0,0,0,0,0,Class_1
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,Class_1


In [123]:
# 타겟변수의 형태를 변환
mapping_dict = {'Class_1' : 1,
                'Class_2' : 2,
                'Class_3' : 3,
                'Class_4' : 4,
                'Class_5' : 5,
                'Class_6' : 6,
                'Class_7' : 7,
                'Class_8' : 8,
                'Class_9' : 9,}
after_mapping_target = data['target'].apply(lambda x : mapping_dict[x])
after_mapping_target[0:4]

0    1
1    1
2    1
3    1
Name: target, dtype: int64

In [124]:
feature_columns = list(data.columns.difference(['target']))
X = data[feature_columns]
y = after_mapping_target
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.2, random_state = 42)
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)

(49502, 93) (12376, 93) (49502,) (12376,)


### 학습 데이터를 에이다부스트 모형에 적합 후 평가데이터로 검증

In [130]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators=100, random_state=0)
clf.fit(train_x, train_y)
pred1 = clf.predict(test_x)
print(accuracy_score(test_y, pred1))

0.6771170006464124


In [132]:
from sklearn.tree import DecisionTreeClassifier

In [133]:
tree_model = DecisionTreeClassifier(max_depth=5)
clf = AdaBoostClassifier(base_estimator=tree_model, n_estimators=10, random_state=0)
clf.fit(train_x, train_y)
pred1 = clf.predict(test_x)
print(accuracy_score(test_y, pred1))

0.6853587588881707


### 추정을 많이 해보자

In [134]:
tree_model = DecisionTreeClassifier(max_depth=5)
clf = AdaBoostClassifier(base_estimator=tree_model, n_estimators=100, random_state=0)
clf.fit(train_x, train_y)
pred1 = clf.predict(test_x)
print(accuracy_score(test_y, pred1))

0.6085972850678733


### 트리의 깊이 늘리기

In [135]:
tree_model = DecisionTreeClassifier(max_depth=20)
clf = AdaBoostClassifier(base_estimator=tree_model, n_estimators=10, random_state=0)
clf.fit(train_x, train_y)
pred1 = clf.predict(test_x)
print(accuracy_score(test_y, pred1))

0.7427278603749192


### 트리의 깊이 최대로 늘리기

In [136]:
tree_model = DecisionTreeClassifier(max_depth=100)
clf = AdaBoostClassifier(base_estimator=tree_model, n_estimators=10, random_state=0)
clf.fit(train_x, train_y)
pred1 = clf.predict(test_x)
print(accuracy_score(test_y, pred1))

0.710245636716225


- 트리의 깊이를 최대로 늘리기 보다는 적당히 늘리는 것이 좋으며,
성능은 이전에 사용했던 Random Forest가 뛰어나다.

### Gradient  Boosting은 Mac M1 환경에서 설치가 되지 않아 다른 운영체제를 통해 사용해볼 예정.

# Stacking

- Meta Learner라고 불리우며, 다양한 모델을 결합하여 사용하는 기법.

# Shap value - 중요변수 추출 방법

### 선형회귀
- '다른 변수가 고정되어있고 TV광고가 1단위 증가할 때, 매출이 0.046 단위 증가한다'
    > 위와 같이 중요변수를 파악하기 쉬움.
- 변수의 유의성은 p-value를 통해 파악 가능.

### Decision Tree
- 독립 변수의 조건에 따라 종속변수를 분리(비가 내린다 -> 축구를 하지 않는다)
- 해석이 매우 용이

- 복잡한 모델은 해석이 쉽지 않음
    - Bagging, RandomForest, Gradient Boosting, NeuralNetwork 등은 모형에 대한 해석과 prediction에 대한 해석이 어려움.
    
    
- Accuracy가 낮고 설명하기 쉬운 모델 - Linear Regression, DT


- Accuracy가 높고 설명하기 어려운 모델 - Ensemble Learning, NN

### Xgboost의 feature importance
- Ensemble learning 모델들은 중요 feature를 추출할 수 있는 알고리즘 내장.


- 예를 들어, 은행 이용 고객 데이터에 대해 수입(Target)이 50만 달러가 넘는지를 예측하는데 중요한 feature 들을 나타냄.
        > Age, Hours per week가 중요한 feature라는 것을 알 수 있음.

### Xgboost의 feature importance 측정 기준
- Weight : 변수 별 데이터를 분리하는 데 쓰인 횟수

    
- Cover : 변수 별 데이터를 분리하는 데 쓰인 횟수(해당 변수로 분리된 데이터의 수로 가중치)
    

- Gain : Feature를 사용했을 때 줄어드는 평균적인 traning loss

    > 이의 문제점은 변수 중요도 측정 기준별로 중요 변수가 상이함.
    
    > 많이 쓰인다 정도만 아는 거지, 양, 음의 방향이냐? 얼마나 영향을 끼치냐? 라는 것의 측정이 불가.

### Feature Importance의 좋고 나쁨의 기준
- Consistency : 특정 feature가 영향이 많이 가도록 모델을 수정하였다면, 중요도 측정 시 해당 feature의 중요도가 줄지 않아야함.
    
    
- Consistency가 없다면, 두 모델의 feature에 대해 비교가 힘들고 feature importance가 높다고 해서 중요하다라고 말하기 힘듦.


- 대부분의 feature importance 지표는 inconsistency

### Shap value
- 연합게임 이론의 Shapley value에서 따옴.
- 전체 payout(지불금, 포상금)에 대한 특정 player에게 기여도에 따른 payout을 배당.